# Description of a problem

We are given three columns of integers with a row for each node. The first two columns contain x and y coordinates of the node positions in a plane. The third column contains node costs. The goal is to select exactly 50% of the nodes (if the number of nodes is odd we round the number of nodes to be selected up) and form a Hamiltonian cycle (closed path) through this set of nodes such that the sum of the total length of the path plus the total cost of the selected nodes is minimized.

The distances between nodes are calculated as Euclidean distances rounded mathematically to
integer values. The distance matrix should be calculated just after reading an instance and then only
the distance matrix (no nodes coordinates) should be accessed by optimization methods to allow
instances defined only by distance matrices.

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import random
from pathlib import Path

import numpy as np
import pandas as pd

from hamiltonian_cycle.algorithms.lab1 import init_random_solution

# from hamiltonian_cycle.algorithms.lab3.objective_change import (
#     objective_change_inter_route,
#     objective_change_two_edges,
#     objective_change_two_nodes,
# )
from hamiltonian_cycle.algorithms.lab3.operations import (
    inter_route_swap,
    two_edges_exchange,
    two_nodes_exchange,
)
from hamiltonian_cycle.costs import dm, function_cost

In [6]:
def read_dataset_csv(csv_path: Path) -> pd.DataFrame:
    return pd.read_csv(csv_path, sep=";", names=["x", "y", "cost"])


DATA_DIR = Path("../data").resolve()

ds_a = read_dataset_csv(DATA_DIR / "TSPA.csv")
ds_b = read_dataset_csv(DATA_DIR / "TSPB.csv")

dm_a = dm(ds_a)
dm_b = dm(ds_b)

In [7]:
best_solution = list(init_random_solution(ds_a, dm_a, 0).index)

In [9]:
best_solution = list(init_random_solution(ds_a, dm_a, 0).index)
ds_a.loc[best_solution]

,x,y,cost
97,2465,426,310
79,1852,682,561
13,3741,252,1839
20,34,1705,1371
95,3691,1650,1075
...,...,...,...
53,2097,299,81
116,883,826,345
59,1172,933,125
150,2540,260,1935


In [11]:
function_cost(ds_a.loc[list(init_random_solution(ds_a, dm_a, 0).index)])

277718

In [46]:
import numpy as np
import pandas as pd


def objective_change_two_nodes(dm: np.ndarray, solution: list, i: int, j: int) -> float:
    if i == j:
        return 0.0

    n = len(solution)
    a, b = solution[i], solution[j]

    a_prev = solution[i - 1] if i > 0 else solution[-1]
    a_next = solution[(i + 1) % n]
    b_prev = solution[j - 1] if j > 0 else solution[-1]
    b_next = solution[(j + 1) % n]

    # Edges to be removed and added
    edges_removed = []
    edges_added = []

    # Remove edges connected to a and b
    if a_prev not in (a, b):
        edges_removed.append((a_prev, a))
        edges_added.append((a_prev, b))
    if a_next not in (a, b):
        edges_removed.append((a, a_next))
        edges_added.append((b, a_next))
    if b_prev not in (a, b):
        edges_removed.append((b_prev, b))
        edges_added.append((b_prev, a))
    if b_next not in (a, b):
        edges_removed.append((b, b_next))
        edges_added.append((a, b_next))

    # Calculate delta
    delta = -sum(dm[u, v] for u, v in edges_removed) + sum(
        dm[u, v] for u, v in edges_added
    )

    return delta


def objective_change_two_edges(dm: np.ndarray, solution: list, i: int, j: int) -> float:
    if i >= j or (i == 0 and j == len(solution) - 1):
        return 0.0

    n = len(solution)
    a_prev = solution[i - 1] if i > 0 else solution[-1]
    a = solution[i]
    b = solution[j]
    b_next = solution[(j + 1) % n]

    # Edges before and after reversal
    cost_before = dm[a_prev, a] + dm[b, b_next]
    cost_after = dm[a_prev, b] + dm[a, b_next]

    delta = cost_after - cost_before

    return delta


def objective_change_inter_route(
    dm: np.ndarray, solution: list, i: int, vacant_node: int, node_costs: list
) -> float:
    n = len(solution)
    node_in_solution = solution[i]
    prev_node = solution[i - 1] if i > 0 else solution[-1]
    next_node = solution[(i + 1) % n]

    # Edge costs before and after the swap
    edge_cost_before = dm[prev_node, node_in_solution] + dm[node_in_solution, next_node]
    edge_cost_after = dm[prev_node, vacant_node] + dm[vacant_node, next_node]

    # Node costs before and after the swap
    node_cost_before = node_costs[node_in_solution]
    node_cost_after = node_costs[vacant_node]

    delta = (edge_cost_after - node_cost_after) - (edge_cost_before - node_cost_before)

    return delta


def two_nodes_exchange(solution: list, i: int, j: int) -> list:
    new_solution = solution.copy()
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution


def two_edges_exchange(solution: list, i: int, j: int) -> list:
    if i >= j:
        return solution.copy()  # No change if i >= j

    new_solution = solution.copy()
    new_solution[i : j + 1] = new_solution[i : j + 1][::-1]
    return new_solution


def inter_route_swap(
    solution: list,
    i: int,
    vacant_node: int,
    selected_nodes: set,
    non_selected_nodes: set,
) -> tuple:
    new_solution = solution.copy()
    node_in_solution = new_solution[i]
    new_solution[i] = vacant_node

    # Update the node sets
    selected_nodes = selected_nodes.copy()
    non_selected_nodes = non_selected_nodes.copy()

    selected_nodes.remove(node_in_solution)
    selected_nodes.add(vacant_node)
    non_selected_nodes.remove(vacant_node)
    non_selected_nodes.add(node_in_solution)

    return new_solution, selected_nodes, non_selected_nodes


def get_remaining_nodes(selected_nodes: set, num_nodes: int) -> set:
    return set(range(num_nodes)) - selected_nodes


def browse_intra_solutions(dm: np.ndarray, solution: list, intra_search: str) -> tuple:
    intra_neighbors = []
    n = len(solution)
    for i in range(n):
        for j in range(i + 1, n):
            if intra_search == "node":
                delta_nodes = objective_change_two_nodes(dm, solution, i, j)
                if delta_nodes < 0:
                    intra_neighbors.append((i, j, delta_nodes, "node"))
            elif intra_search == "edge":
                delta_edges = objective_change_two_edges(dm, solution, i, j)
                if delta_edges < 0:
                    intra_neighbors.append((i, j, delta_edges, "edge"))
    return intra_neighbors


def browse_inter_solutions(
    dm: np.ndarray, solution: list, non_selected_nodes: set, costs: list
) -> list:
    inter_neighbors = []
    for i in range(len(solution)):
        for vacant_node in non_selected_nodes:
            inter_delta = objective_change_inter_route(
                dm, solution, i, vacant_node, costs
            )
            if inter_delta < 0:
                inter_neighbors.append((i, vacant_node, inter_delta, "inter"))
    return inter_neighbors


def update_solution(
    solution: list,
    best_neighbor: tuple,
    selected_nodes: set,
    non_selected_nodes: set,
) -> tuple:
    neighbor_type = best_neighbor[-1]

    if neighbor_type == "node":
        i, j = best_neighbor[:2]
        solution = two_nodes_exchange(solution, int(i), int(j))
    elif neighbor_type == "edge":
        i, j = best_neighbor[:2]
        solution = two_edges_exchange(solution, int(i), int(j))
    elif neighbor_type == "inter":
        i, vacant_node = best_neighbor[:2]
        solution, selected_nodes, non_selected_nodes = inter_route_swap(
            solution, int(i), int(vacant_node), selected_nodes, non_selected_nodes
        )
    return solution, selected_nodes, non_selected_nodes


def local_search(
    ds: pd.DataFrame,
    dm: np.ndarray,
    initial_solution: list,
    strategy: str = "greedy",
    intra_search: str = "edge",
    debug_mode: bool = True,
) -> pd.DataFrame:
    num_nodes = len(dm)
    initial_cost = function_cost(ds.loc[initial_solution])

    solution = initial_solution.copy()
    selected_nodes = set(initial_solution)
    non_selected_nodes = get_remaining_nodes(selected_nodes, num_nodes)

    while True:
        intra_neighbors = browse_intra_solutions(dm, solution)
        inter_neighbors = browse_inter_solutions(
            dm, solution, non_selected_nodes, ds["cost"].tolist()
        )

        all_neighbors = intra_neighbors + inter_neighbors

        if not all_neighbors:
            # No improvement found
            break

        if strategy == "greedy":
            random.shuffle(all_neighbors)
            best_neighbor = next((n for n in all_neighbors if n[2] < 0), None)
        elif strategy == "steepest":
            best_neighbor = min(all_neighbors, key=lambda x: x[2])

        if best_neighbor is None:
            # No improving neighbor found
            break

        # Save the old solution for debugging
        old_solution = solution.copy()

        # Update solution and cost
        solution, selected_nodes, non_selected_nodes = update_solution(
            solution, best_neighbor, selected_nodes, non_selected_nodes
        )
        initial_cost += best_neighbor[2]

        if debug_mode:
            # Calculate real improvement
            real_improvement = function_cost(ds.loc[old_solution]) - function_cost(
                ds.loc[solution]
            )

            if real_improvement != -best_neighbor[2]:
                print(f"Promised improvement: {best_neighbor[2]}")
                print(f"Real improvement: {real_improvement}")
                print(f"Operation: {best_neighbor[-1]}")
                print("===========")

            assert function_cost(ds.loc[solution]) < function_cost(ds.loc[old_solution])

    return ds.loc[solution]


from hamiltonian_cycle.algorithms.lab1 import init_greedy_cycle
from hamiltonian_cycle.experiment import perform_experiment


def init_local_search(
    ds: pd.DataFrame,
    dm: np.ndarray,
    start: int,
    strategy: str,
    intra_search: str,
    debug_mode: bool = True,
    algo_to_enchance: str = "greedy_cycle",
) -> pd.DataFrame:
    if algo_to_enchance == "greedy_cycle":
        solution = list(init_greedy_cycle(ds, dm, start).index)
    elif algo_to_enchance == "random":
        solution = list(init_random_solution(ds, dm, start).index)
    solution = local_search(ds, dm, solution, strategy, intra_search, debug_mode)
    return solution

In [49]:
algos = ["greedy_cycle", "random"]
intra_search = ["node", "edge"]
strategy = ["greedy", "steepest"]

#### Results on Dataset A

In [51]:
for algo in algos:
    for search in intra_search:
        for strat in strategy:
            perform_experiment(
                ds_a,
                dm_a,
                f"Algorithm: {algo}, Intra search: {search}, Strategy: {strat}",
                init_local_search,
                **{
                    "strategy": strat,
                    "intra_search": search,
                    "debug_mode": True,
                    "algo_to_enchance": algo,
                },
            )

TypeError: browse_intra_solutions() missing 1 required positional argument: 'intra_search'

In [26]:
for algo in algos:
    for search in intra_search:
        for strat in strategy:
            perform_experiment(
                ds_a,
                dm_a,
                f"Algorithm: {algo}, Intra search: {search}, Strategy: {strat}",
                init_local_search,
                **{
                    "strategy": strat,
                    "intra_search": search,
                    "debug_mode": True,
                    "algo_to_enchance": algo,
                },
            )

71057

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)

In [ ]:
perform_experiment(
    ds_a,
    dm_a,
    "Dataset A",
    init_local_search,
    **{
        "strategy": "greedy",
        "intra_search": "node",
        "debug_mode": True,
        "algo_to_enchance": "greedy_cycle",
    },
)